# Deep Learning
## Project_3

### - Task #1

In [1]:
"""
  Autocompletion of the last character of words
  Given the first three letters of a four-letters word, learn to predict the last letter 
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
import numpy as np


vocab = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
         'h', 'i', 'j', 'k', 'l', 'm', 'n',
         'o', 'p', 'q', 'r', 's', 't', 'u',
         'v', 'w', 'x', 'y', 'z']

# index array of characters in vocab
v_map = {n: i for i, n in enumerate(vocab)}
v_len = len(v_map)

# training data (character sequences)
# wor -> X, d -> Y
# woo -> X, d -> Y
training_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
test_data = ['wood', 'deep', 'cold', 'load', 'love', 'dear', 'dove', 'cell', 'life', 'keep']

def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        # Indices of the first three alphabets of the words
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
        input = [v_map[n] for n in seq[:-1]]
        # Indices of the last alphabet of the words
        # 3, 3, 15, 4, 3 ...
        target = v_map[seq[-1]]

        # One-hot encoding of the inputs into the sequences of 26-dimensional vectors
        # [0, 1, 2] ==>
        # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
        input_batch.append(np.eye(v_len)[input])
        
        # We don't apply one-hot encoding for the output,  
        # since we'll use sparse_softmax_cross_entropy_with_logits
        # as our loss function
        target_batch.append(target)

    return input_batch, target_batch


learning_rate = 0.01
n_hidden = 10
total_epoch = 100
n_step = 3 # the length of the input sequence
n_input = n_class = v_len # the size of each input

"""
  Phase 1: Create the computation graph
"""
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

#                output (pred. of forth letter)
#                 | (W,b)
#                outputs (hidden)   
#       |    |    | 
# RNN: [t1]-[t2]-[t3]
#       x1   x2   x3

# Create an LSTM cell
cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# Apply dropout for regularization
#cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.75)

# Create the RNN
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
# outputs : [batch_size, max_time, cell.output_size]

# Transform the output of RNN to create output values
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
# [batch_size, cell.output_size]
model = tf.matmul(outputs, W) + b
# [batch_size, n_classes]

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

"""
  Phase 2: Train the model
"""
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    input_batch, target_batch = make_batch(training_data)

    for epoch in range(total_epoch):
        _, loss = sess.run([optimizer, cost],
                           feed_dict={X: input_batch, Y: target_batch})

        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

    print('Optimization finished')

    """
      Make predictions
    """
    seq_data = training_data # test_data
    prediction = tf.cast(tf.argmax(model, 1), tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, Y), tf.float32))

    input_batch, target_batch = make_batch(seq_data)

    predict, accuracy_val = sess.run([prediction, accuracy],
                                     feed_dict={X: input_batch, Y: target_batch})

    predicted = []
    for idx, val in enumerate(seq_data):
        last_char = vocab[predict[idx]]
        predicted.append(val[:3] + last_char)

    print('\n=== Predictions ===')
    print('Input:', [w[:3] + ' ' for w in seq_data])
    print('Predicted:', predicted)
    print('Accuracy:', accuracy_val)

Epoch: 0001 cost = 3.608380
Epoch: 0002 cost = 3.513700
Epoch: 0003 cost = 3.419682
Epoch: 0004 cost = 3.325208
Epoch: 0005 cost = 3.229174
Epoch: 0006 cost = 3.130659
Epoch: 0007 cost = 3.028875
Epoch: 0008 cost = 2.923135
Epoch: 0009 cost = 2.812870
Epoch: 0010 cost = 2.697656
Epoch: 0011 cost = 2.577245
Epoch: 0012 cost = 2.451602
Epoch: 0013 cost = 2.320984
Epoch: 0014 cost = 2.186035
Epoch: 0015 cost = 2.047878
Epoch: 0016 cost = 1.908164
Epoch: 0017 cost = 1.769062
Epoch: 0018 cost = 1.633132
Epoch: 0019 cost = 1.503088
Epoch: 0020 cost = 1.381399
Epoch: 0021 cost = 1.269831
Epoch: 0022 cost = 1.169086
Epoch: 0023 cost = 1.078635
Epoch: 0024 cost = 0.996863
Epoch: 0025 cost = 0.921522
Epoch: 0026 cost = 0.850343
Epoch: 0027 cost = 0.781618
Epoch: 0028 cost = 0.714549
Epoch: 0029 cost = 0.649317
Epoch: 0030 cost = 0.586881
Epoch: 0031 cost = 0.528626
Epoch: 0032 cost = 0.475938
Epoch: 0033 cost = 0.429831
Epoch: 0034 cost = 0.390670
Epoch: 0035 cost = 0.358069
Epoch: 0036 cost = 0

### - Task #2

In [2]:
tf.reset_default_graph()

In [3]:
"""
  Autocompletion of the last character of words
  Given the first three letters of a four-letters word, learn to predict the last letter
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
import numpy as np


vocab = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
         'h', 'i', 'j', 'k', 'l', 'm', 'n',
         'o', 'p', 'q', 'r', 's', 't', 'u',
         'v', 'w', 'x', 'y', 'z']

# index array of characters in vocab
v_map = {n: i for i, n in enumerate(vocab)}
v_len = len(v_map)

# training data (character sequences)
# wor -> X, d -> Y
# woo -> X, d -> Y
training_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
test_data = ['wood', 'deep', 'cold', 'load', 'love', 'dear', 'dove', 'cell', 'life', 'keep']

def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        # Indices of the first three alphabets of the words
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
        input = [v_map[seq[0]], v_map[seq[1]], v_map[seq[-1]]]
        # Indices of the last alphabet of the words
        # 3, 3, 15, 4, 3 ...
        target = v_map[seq[-2]]

        # One-hot encoding of the inputs into the sequences of 26-dimensional vectors
        # [0, 1, 2] ==>
        # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
        input_batch.append(np.eye(v_len)[input])

        # We don't apply one-hot encoding for the output,
        # since we'll use sparse_softmax_cross_entropy_with_logits
        # as our loss function
        target_batch.append(target)

    return input_batch, target_batch


learning_rate = 0.01
n_hidden = 10
total_epoch = 100
n_step = 3 # the length of the input sequence
n_input = n_class = v_len # the size of each input

"""
  Phase 1: Create the computation graph
"""
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

#                output (pred. of forth letter)
#                 | (W,b)
#                outputs (hidden)
#       |    |    |
# RNN: [t1]-[t2]-[t3]
#       x1   x2   x3

# Create an LSTM cell
cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# Apply dropout for regularization
#cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=0.75)

# Create the RNN
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
# outputs : [batch_size, max_time, cell.output_size]

# Transform the output of RNN to create output values
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
# [batch_size, cell.output_size]
model = tf.matmul(outputs, W) + b
# [batch_size, n_classes]

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

"""
  Phase 2: Train the model
"""
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    input_batch, target_batch = make_batch(training_data)

    for epoch in range(total_epoch):
        _, loss = sess.run([optimizer, cost],
                           feed_dict={X: input_batch, Y: target_batch})

        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

    print('Optimization finished')

    """
      Make predictions
    """
    seq_data = training_data # test_data
    prediction = tf.cast(tf.argmax(model, 1), tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, Y), tf.float32))

    input_batch, target_batch = make_batch(seq_data)

    predict, accuracy_val = sess.run([prediction, accuracy],
                                     feed_dict={X: input_batch, Y: target_batch})

    predicted = []
    for idx, val in enumerate(seq_data):
        middle_char = vocab[predict[idx]]
        predicted.append(val[:2] + middle_char + val[-1])

    print('\n=== Predictions ===')
    print('Input:', ["{}_{}".format(w[:2], w[-1]) for w in seq_data])
    print('Predicted:', predicted)
    print('Accuracy:', accuracy_val)

Epoch: 0001 cost = 3.438918
Epoch: 0002 cost = 3.365723
Epoch: 0003 cost = 3.292951
Epoch: 0004 cost = 3.219447
Epoch: 0005 cost = 3.144657
Epoch: 0006 cost = 3.068118
Epoch: 0007 cost = 2.989360
Epoch: 0008 cost = 2.907930
Epoch: 0009 cost = 2.823430
Epoch: 0010 cost = 2.735563
Epoch: 0011 cost = 2.644189
Epoch: 0012 cost = 2.549383
Epoch: 0013 cost = 2.451471
Epoch: 0014 cost = 2.351079
Epoch: 0015 cost = 2.249162
Epoch: 0016 cost = 2.147007
Epoch: 0017 cost = 2.046167
Epoch: 0018 cost = 1.948310
Epoch: 0019 cost = 1.854992
Epoch: 0020 cost = 1.767367
Epoch: 0021 cost = 1.685909
Epoch: 0022 cost = 1.610252
Epoch: 0023 cost = 1.539263
Epoch: 0024 cost = 1.471398
Epoch: 0025 cost = 1.405230
Epoch: 0026 cost = 1.339834
Epoch: 0027 cost = 1.274840
Epoch: 0028 cost = 1.210264
Epoch: 0029 cost = 1.146308
Epoch: 0030 cost = 1.083337
Epoch: 0031 cost = 1.022002
Epoch: 0032 cost = 0.963304
Epoch: 0033 cost = 0.908422
Epoch: 0034 cost = 0.858272
Epoch: 0035 cost = 0.813050
Epoch: 0036 cost = 0